In [ ]:
from skimage import segmentation, io
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import calinski_harabasz_score, silhouette_score
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
import pandas as pd
from utils.utils import generate_synthetic_image, plot_dendrogram
from utils.image_manager import ImagesManager
from utils.mask_manager import MaskManager
from utils.features_manager import FeaturesManager
import os
import warnings
plt.rcParams["axes.grid"] = False
from sklearn.preprocessing import scale
from quickshift_tabular.quickshift_tabular import quickshift_tab

In [ ]:
model = keras.models.load_model('./model/model.h5')

In [ ]:
#path_to_images = './data/eval_xai_method/dog_resized'
path_to_images ='./data/debug'
all_images = os.listdir(path_to_images)
all_images = [x for x in all_images if x != '.DS_Store']

In [ ]:
all_images

In [ ]:
num_images = len(all_images)

In [ ]:
images_chosen = np.random.choice(all_images, size=num_images, replace=False)

In [ ]:
path_to_images='./data/debug'
images_extension='jpg'
normalise_features=True
num_channels=3
kernel_size=4
max_dist=20
ratio=.2
kernel_size_tab=1
max_dist_tab=0.3
ratio_tab=1


images_manager = ImagesManager(location=path_to_images, images_extension=images_extension)
#images_names = images_manager.get_images_names()
images_names = ['resized_dog.6.jpg']
dict_masks = {}
spatial_columns = ['mass_center0', 'mass_center1']

for image_name in images_names:
    masks_manager = MaskManager(location=path_to_images, image_name=image_name)
    mask = masks_manager.get_mask(kernel_size=kernel_size, max_dist=max_dist, ratio=ratio)
    dict_masks[image_name] = mask

    features_manager = FeaturesManager(location=path_to_images, image_name=image_name, 
                                       mask=mask, normalise_features=True, num_channels=num_channels)
    df_features, deleted_superpixels = features_manager.get_features()
    col_names = df_features.columns.tolist()
    col_reodered = spatial_columns + [x for x in col_names if x not in spatial_columns]
    df_features = df_features.reindex(columns=col_reodered)
    mask_list = [x[0] for x in df_features.index.tolist()]

    X = df_features.to_numpy()
    
    mask_tabular_quickshift_list = quickshift_tab(X=X, ratio=ratio_tab, 
                                             kernel_size=kernel_size_tab, max_dist=max_dist_tab)
    mask_tabluar_quickshift = np.empty(shape=mask.shape)
    
    for original_value, new_value in zip(mask_list, mask_tabular_quickshift_list):
        idx = np.where(mask == original_value)
        mask_tabluar_quickshift[idx] = new_value
    
    image_path = os.path.join(path_to_images, image_name)
    img = io.imread(image_path)
    plt.imshow(img)
    plt.show()
    
    plt.imshow(mask_tabluar_quickshift)
    plt.show()
    for sp_value in np.unique(mask_tabular_quickshift_list):
        mk = mask_tabluar_quickshift == sp_value
        mk = mk[..., np.newaxis]
        
        img_occluded = img * mk
        plt.imshow(img_occluded, vmin=np.min(img_occluded), vmax=np.max(img_occluded))
        plt.show()
        

In [ ]:
df_features.head()

In [ ]:
mask

In [ ]:
id_0 = np.where(mask==0)
id_1 = np.where(mask==1)
print(id_0)
print(id_1)

# Trash